Строим запрос к нашему сервису

In [1]:
import requests

import urllib.request
import json 

In [4]:
# формируем запрос
def send_json(x):
    cont_transformer, diameter_max, relative_velocity, miss_distance, absolute_magnitude, sentry = x
#                       ('est_diameter_min', cont_transformer),
#                       ('est_diameter_max', diameter_max),
#                       ('relative_velocity', relative_velocity),
#                       ('miss_distance', miss_distance),
#                       ('absolute_magnitude', absolute_magnitude),
#                       ('sentry_object', sentry)
    #print(description, company_profile, benefits)
#  ({"est_diameter_min": [cont_transformer],
#                                               "est_diameter_max": [diameter_max],
#                                               "relative_velocity": [relative_velocity],
#                                               "sentry_object": [bool(sentry)],
#                                              'miss_distance':[miss_distance],
#                                              'absolute_magnitude':[absolute_magnitude]   
    body = {
#        'name': description, 
        'est_diameter_min': cont_transformer,
        'est_diameter_max': diameter_max,
        'relative_velocity': relative_velocity,
        'miss_distance': miss_distance,
        'absolute_magnitude': absolute_magnitude,
        'sentry_object': bool(sentry)
        }
    myurl = 'http://127.0.0.1:5000' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [5]:
# # формируем запрос
# def send_json(x):
#     cont_transformer, diameter_max, absolute_magnitude, relative_velocity = x
# #    print(description, company_profile, benefits)
#     body = {
#         'est_diameter_min': [cont_transformer],
#         'est_diameter_max': [diameter_max],
#         'absolute_magnitude': [absolute_magnitude],
# #        'sentry_object': [sentry],
#         'relative_velocity': [relative_velocity]
#         }
#     myurl = 'http://127.0.0.1:5000' + '/predict'
#     headers = {'content-type': 'application/json; charset=utf-8'}
#     response = requests.post(myurl, json=body, headers=headers)
#     return response.json()['predictions']

In [6]:
# print(cont_transformer, diameter_max, absolute_magnitude, relative_velocity)

In [7]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
# response = send_json(data)
# print('предсказание', response)

Сделаем обработку массы запросов:

1. загрузим данные X_test

https://drive.google.com/file/d/1-Ltk-t1acAoA5Xtz0zPZ1yOqjoYiP2h5

https://drive.google.com/file/d/1vb0KLnOeRsg4G33TaPxKEV2OMnla1oyL

In [8]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse


X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

2. Передаем запрос одиночный из наших данных и посмотрим на ответ

In [9]:
response = send_json(X_test[['est_diameter_min', 'est_diameter_max', 'relative_velocity', 'miss_distance', 'absolute_magnitude', 'sentry_object']].iloc[0,:])

In [10]:
response

0.0

In [11]:
X_test[['est_diameter_min', 'est_diameter_max', 'relative_velocity', 'miss_distance', 'absolute_magnitude', 'sentry_object']].head()

,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,sentry_object
0,0.024241,0.054205,22148.962596,5.028574e+07,25.20,False
1,0.012722,0.028447,26477.211836,1.683201e+06,26.60,False
2,0.013322,0.029788,33770.201397,3.943220e+06,26.50,False
3,0.193444,0.432554,43599.575296,7.346837e+07,20.69,False
4,0.044112,0.098637,36398.080883,6.352916e+07,23.90,False


3. Сделаем N запросов и оценим время

In [12]:
N = 50

In [13]:
%%time
predictions = X_test[
                     ['est_diameter_min', 'est_diameter_max', 'relative_velocity', 'miss_distance', 'absolute_magnitude', 'sentry_object']
                     ].iloc[:N].apply(lambda x: send_json(x), axis=1)

Wall time: 1.17 s


In [14]:
predictions.values[:50]

array([0.  , 0.  , 0.  , 0.03, 0.  , 0.47, 0.  , 0.  , 0.01, 0.15, 0.41,
       0.03, 0.62, 0.  , 0.  , 0.  , 0.15, 0.02, 0.  , 0.59, 0.  , 0.  ,
       0.18, 0.08, 0.25, 0.31, 0.97, 0.32, 0.04, 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.51, 0.  , 0.  , 0.11, 0.  , 0.  , 0.19,
       0.  , 0.02, 0.04, 0.  , 0.  , 0.02])

4. Посчитаем метрику

In [15]:
y_test.head()

,hazardous
0,False
1,False
2,False
3,False
4,False


In [16]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.41, F-Score=0.615, Precision=0.667, Recall=0.571


In [17]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.9202657807308969